# Lab 09
### IS6713

Instructions:
1. Add the lines of code required to obtain the requested output.

2. The code should be inserted where requested   
   
3. Do not modify other parts of this notebook.  
  
4. Remember to write in your name and abc123 in the first cell of this notebook.
  
5. Upload the notebook on Blackboard before the deadline!

Name: Ben Hoffman

abc123: peo042

## Exercise 9.1
A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").  
The data I am providing is a reformatted version of the Kaggle one. For example, "neutral" Tweets are not included for simplicity. The dataset includes three fields: **airline**, **full_text**, and **gold_label**.  
Tasks:
- Load the **w9_airlinetwitter.jsonl** dataset
- Vectorize using the TFIDF approach
- Split the dataset in train (80%) and test (20%) sets
- Train the RandomForest Classifier over the train set
- Predict the labels of the test set using the trained model
- Calculate and print:
  - Classification report
  - Confusion matrix
  - Accuracy score

In [1]:
### Step 1: load the dataset
## Don't modify this code
import json

dataset = []
with open('w9_airlinetwitter.jsonl') as infile:
    for line in infile:
        dataset.append(json.loads(line.strip()))

In [2]:
assert(len(dataset)==11541)
print("Dataset Correctly Loaded")

Dataset Correctly Loaded


In [3]:
## Check the format (don't modify this cell)
## It should be something like:
#"""{'airline': 'Virgin America',
# 'full_text': "@VirginAmerica plus you've added commercials to the experience... tacky.",
# 'gold_label': 'positive'}
# """

dataset[0]

{'airline': 'Virgin America',
 'full_text': "@VirginAmerica plus you've added commercials to the experience... tacky.",
 'gold_label': 'positive'}

### Analysis Section

In [16]:
## Step 2 - Vectorize the dataset
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer as TV

def nopun(text):
    no_pun = "".join([c for c in text if c not in string.punctuation])
    return no_pun

goldlab, feature = [], []

for d in dataset:
    text = d['full_text'].lower()
    token = nltk.word_tokenize(text)
    
    stop = set(stopwords.words('english'))
    cleantok = [t for t in token if len(t.lower())>1 and (t.lower() not in stop)]
        
    clean = []
    for w in cleantok:
        cleant = nopun(w)
        if len(cleant.strip())>1:
            clean.append(cleant.strip())
            
    goldlab.append(d['gold_label'])
    feature.append(' '.join(clean))

vector = TV(max_features = 2500, min_df = 7, max_df = 0.8, stop_words = stopwords.words('english'))

processedvec = vector.fit_transform(feature).toarray()

virginamerica plus ve added commercials experience tacky


In [17]:
feature[0]

'virginamerica plus ve added commercials experience tacky'

In [5]:
## Step 3 - Split in train and test
from sklearn.model_selection import train_test_split as tts

xtrain, xtest, ytrain, ytest = tts(processedvec, goldlab, test_size = .2, random_state = 0)

In [6]:
## Step 4: train the classifier
from sklearn.ensemble import RandomForestClassifier as RFC

textclass = RFC(n_estimators = 200, random_state = 0)
textclass.fit(xtrain, ytrain)

RandomForestClassifier(n_estimators=200, random_state=0)

In [7]:
## Step 5: Predict the labels
pred = textclass.predict(xtest)

In [8]:
## Step 6: Assess the performance (Accuracy, Confusion Matrix, Classification Report)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

tn, fp, fn, tp = confusion_matrix(ytest, pred).ravel()

print("TP", tp, "TN", tn, "FP", fp, "FN", fn)
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))
print('RF Accuracy: {0:.2f}'.format(accuracy_score(ytest, pred)))

TP 278 TN 1805 FP 55 FN 171
[[1805   55]
 [ 171  278]]
              precision    recall  f1-score   support

    negative       0.91      0.97      0.94      1860
    positive       0.83      0.62      0.71       449

    accuracy                           0.90      2309
   macro avg       0.87      0.79      0.83      2309
weighted avg       0.90      0.90      0.90      2309

RF Accuracy: 0.90


# Exercise 9.2 
### IRIS
Do the following:
1. Load the **iris.csv** dataset into a numpy array. The first 4 columns are the  features/attributes. The last column is the class. Simply load the class as a list of strings. You can use the CVS method to load the features into a list of lists (X) and to load the gold label as a list (y). Then, convert X and y into a numpy array (hint: use np.array() for the conversion).
2. Use train_test_split() to split X and y into train and test. (use 0.2 for the test_size)
3. Train an SVM classifier on the train split and evaluate its accuracy on the test split. Print the accuracy.
4. Apply PCA to the array X, reducing the number of features to 2. Save the new set of features in a different array (e.g. newX)
4. Split the new dataset (newX) and y in train and test
5. Train an SVM classifier on the train split and evaluate the accuracy on the test. Print the new accuracy.
6. Compare the results from 3) and 5). Which one is better?
  
Note that this is a toy dataset, so all scores will be high.

In [9]:
import csv
import numpy as np
X = [] # Will be a list of lists
y = [] # will be a list

## Write here your code
## read the file
with open('iris.csv') as file:
    for row in file:
        column = row.strip().split(',')
        X.append([float(x) for x in column[:-1]])
        y.append(column[-1])

X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split as tts
# split into train and test
xtrain, xtest, ytrain, ytest = tts(X,y,test_size = .2, random_state= 0)

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# train an SVM Classifier, test and print the accuracy
clf = SVC()
clf.fit(xtrain, ytrain)

pred = clf.predict(xtest)
accuracy = accuracy_score(ytest, pred)
print("The model accuracy is {}".format(accuracy))

The model accuracy is 1.0


In [12]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Run a PCA to select only 2 components and create a new dataset
ss = StandardScaler()
xscale = ss.fit_transform(X)
pca = PCA(n_components = 2)
newx = pca.fit_transform(xscale)

In [13]:
from sklearn.model_selection import train_test_split as tts

## Split the new dataset, train a SVM classifier, test and print the accuracy
xtrain, xtest, ytrain, ytest = tts(newx,y,test_size = .2, random_state= 0)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(xtrain, ytrain)

pred = clf.predict(xtest)
accuracy = accuracy_score(ytest, pred)
print("The model accuracy is {}".format(accuracy))

The model accuracy is 0.9


### Comment about the differences. Is the difference big or small?

In [14]:
""" 
Comment here

In this case, it is a big difference because it goes from 1.0 accuracy to a non-perfect (.9) accuracy.

Although that is a big difference, .9 accuracy is still a very good score. Thus, does not exclude it from
contention in the model building process.


"""

' \nComment here\n\nIn this case, it is a big difference because it goes from 1.0 accuracy to a non-perfect (.9) accuracy.\n\nAlthough that is a big difference, .9 accuracy is still a very good score. Thus, does not exclude it from\ncontention in the model building process.\n\n\n'